# Assignment 3 - Problem 4
## DNN with 2 Hidden Layers and input_dim*2 Units

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib as plt
from six.moves import cPickle as pickle

First reload the data we generated in notmist.ipynb.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

### Pre-process dataset for DNN

In [6]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


# Make Model Using:
## 1 - L2 - Regularization
## 2 - Dropout

## 1 - L2 Regularization
### Hyper - Parameter Tuning
#### 1 - Initial Learning Rate - Done in the training step

#### 2 - Learning Rate Decay 
#### 3 - Momentum

##### All of the above hyper-parameter tuning is handled by Adagrad

In [59]:
batch_size = 128
H = image_size*image_size * 2 # No. of hidden units = Flattened Dimension of input * 2

from math import sqrt

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    # Training set
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # Validation set
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    # Test set
    tf_test_dataset = tf.constant(test_dataset)
    # Regularization parameter - beta 
    beta = tf.placeholder(tf.float32)
    # Learning rate - alpha
    alpha = 0.5
    # Momentum initialization
    momentum = 0
    
    # Variables. (used stddev=sqrt(2/(length_of_flattened_matrix) --> This avoids diverges of weights 
    #                                                                  which doesn't cause NaN values 
    #                                                                   to occure in ReLU vectors)) 
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, H], 
                                             stddev=sqrt(3.0/(image_size*image_size + H))))
    biases1 = tf.Variable(tf.zeros([H]))
    weights2 = tf.Variable(tf.truncated_normal([H, H],
                                            stddev=sqrt(3.0/(H + H))))
    biases2 = tf.Variable(tf.zeros([H]))
    weights3 = tf.Variable(tf.truncated_normal([H, H],
                                            stddev=sqrt(3.0/(H + H))))
    biases3 = tf.Variable(tf.zeros([H]))
    weights4 = tf.Variable(tf.truncated_normal([H, num_labels],
                                            stddev=sqrt(3.0/(H + num_labels))))
    biases4 = tf.Variable(tf.zeros([num_labels]))

    # Regularization term  
    regularizer = tf.nn.l2_loss(weights1) +  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4)

    # Training computation.
    train_hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    train_hidden2 = tf.nn.relu(tf.matmul(train_hidden1, weights2) + biases2)
    train_hidden3 = tf.nn.relu(tf.matmul(train_hidden2, weights3) + biases3)
    train_logits = tf.matmul(train_hidden3, weights4) + biases4

    # Training loss
    train_loss = tf.nn.softmax_cross_entropy_with_logits(train_logits, tf_train_labels)
    momentum = 0.9*momentum + (train_loss + beta*regularizer)
    train_l2_loss = tf.reduce_mean(momentum)

    # Validation computation.
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weights2) + biases2)
    valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, weights3) + biases3)
    valid_logits = tf.matmul(valid_hidden3, weights4) + biases4

    # Validation loss
    valid_loss = tf.nn.softmax_cross_entropy_with_logits(valid_logits, tf_valid_labels)
    valid_l2_loss = tf.reduce_mean(valid_loss + beta*regularizer) 

    # Test computation.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weights2) + biases2)
    test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, weights3) + biases3)
    test_logits = tf.matmul(test_hidden3, weights4) + biases4

    # Optimizer - Exponential Learning Rate Decay.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(alpha, 
                                               global_step, 
                                               decay_steps=10000,
                                               decay_rate=0.001)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(train_l2_loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

### Train model - L2 Regularization

In [61]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 0.001}
        _, l, predictions, a = session.run(
            [optimizer, train_l2_loss, train_prediction, learning_rate], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("----------------> ALPHA =", learning_rate.eval())
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%\n\n" % accuracy(test_prediction.eval(), test_labels))

Initialized
----------------> ALPHA = 0.499655
Minibatch loss at step 0: 4.768718
Minibatch accuracy: 10.2%
Validation accuracy: 18.1%
Test accuracy: 19.4%


----------------> ALPHA = 0.466305
Minibatch loss at step 100: 2.796176
Minibatch accuracy: 80.5%
Validation accuracy: 82.7%
Test accuracy: 90.0%


----------------> ALPHA = 0.435181
Minibatch loss at step 200: 2.538755
Minibatch accuracy: 85.2%
Validation accuracy: 83.5%
Test accuracy: 90.4%


----------------> ALPHA = 0.406135
Minibatch loss at step 300: 2.456173
Minibatch accuracy: 79.7%
Validation accuracy: 84.5%
Test accuracy: 91.2%


----------------> ALPHA = 0.379027
Minibatch loss at step 400: 2.194288
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Test accuracy: 92.5%


----------------> ALPHA = 0.353728
Minibatch loss at step 500: 1.986152
Minibatch accuracy: 88.3%
Validation accuracy: 86.2%
Test accuracy: 92.9%


----------------> ALPHA = 0.330119
Minibatch loss at step 600: 2.049335
Minibatch accuracy: 84.4%
Vali

### The above 3 Hidden layered DNN gave 95.8 % accuracy.

## Make a Model - Dropout (as regularization technique)

In [89]:
batch_size = 128
H = image_size*image_size * 2 # No. of hidden units = Flattened Dimension of input * 2

from math import sqrt

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    # Training set
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    # Validation set
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    # Test set
    tf_test_dataset = tf.constant(test_dataset)
    # Learning rate - alpha
    alpha = 0.5
    # Regularization parameter - beta 
    beta = tf.placeholder(tf.float32)
    # Momentum initialization
    momentum = 0
    # Keeping probability while dropping out units
    dropout = 1 
    
    # Variables. (used stddev=sqrt(2/(length_of_flattened_matrix) --> This avoids diverges of weights 
    #                                                                  which doesn't cause NaN values 
    #                                                                   to occure in ReLU vectors)) 
    weights1 = tf.Variable(tf.truncated_normal([image_size*image_size, H], 
                                             stddev=sqrt(3.0/(image_size*image_size + H))))
    biases1 = tf.Variable(tf.zeros([H]))
    weights2 = tf.Variable(tf.truncated_normal([H, H],
                                            stddev=sqrt(3.0/(H + H))))
    biases2 = tf.Variable(tf.zeros([H]))
    weights3 = tf.Variable(tf.truncated_normal([H, H],
                                            stddev=sqrt(3.0/(H + H))))
    biases3 = tf.Variable(tf.zeros([H]))
    weights4 = tf.Variable(tf.truncated_normal([H, H],
                                            stddev=sqrt(3.0/(H + H))))
    biases4 = tf.Variable(tf.zeros([H]))
    weights5 = tf.Variable(tf.truncated_normal([H, num_labels],
                                            stddev=sqrt(3.0/(H + num_labels))))
    biases5 = tf.Variable(tf.zeros([num_labels]))
    
    # Regularization term  
    regularizer = tf.nn.l2_loss(weights1) +  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) + tf.nn.l2_loss(weights4) + tf.nn.l2_loss(weights5)

    # Training computation.
    train_hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    train_hidden2 = tf.nn.relu(tf.matmul(train_hidden1, weights2) + biases2)
    train_hidden3 = tf.nn.relu(tf.matmul(train_hidden2, weights3) + biases3)
    train_hidden4 = tf.nn.relu(tf.matmul(train_hidden3, weights4) + biases4)
    train_logits = tf.matmul(train_hidden4, weights5) + biases5

    # Training loss
    train_loss = tf.nn.softmax_cross_entropy_with_logits(train_logits, tf_train_labels)
    momentum = 0.9*momentum + (train_loss + beta*regularizer)
    train_l2_loss = tf.reduce_mean(momentum)

    # Validation computation.
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, weights2) + biases2)
    valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, weights3) + biases3)
    valid_hidden4 = tf.nn.relu(tf.matmul(valid_hidden3, weights4) + biases4)
    valid_logits = tf.matmul(valid_hidden4, weights5) + biases5

    # Validation loss
    valid_loss = tf.nn.softmax_cross_entropy_with_logits(valid_logits, tf_valid_labels)
    valid_l2_loss = tf.reduce_mean(valid_loss) 

    # Test computation.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, weights2) + biases2)
    test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, weights3) + biases3)
    test_hidden4 = tf.nn.relu(tf.matmul(test_hidden3, weights4) + biases4)
    test_logits = tf.matmul(test_hidden4, weights5) + biases5

    # Optimizer - Exponential Learning Rate Decay.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(alpha, 
                                               global_step, 
                                               decay_steps=75000,
                                               decay_rate=0.001)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(train_l2_loss, global_step=global_step)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_logits)
    valid_prediction = tf.nn.softmax(valid_logits)
    test_prediction = tf.nn.softmax(test_logits)

### Train the model - L2 Regularization

In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 0.001}
        _, l, predictions, a = session.run(
            [optimizer, train_l2_loss, train_prediction, learning_rate], feed_dict=feed_dict)
        if (step % 100 == 0):
            print("----------------> ALPHA =", learning_rate.eval())
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%\n\n" % accuracy(test_prediction.eval(), test_labels))

Initialized
----------------> ALPHA = 0.499954
Minibatch loss at step 0: 5.679862
Minibatch accuracy: 8.6%
Validation accuracy: 21.2%
Test accuracy: 22.4%


----------------> ALPHA = 0.49537
Minibatch loss at step 100: 3.715647
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Test accuracy: 88.3%


----------------> ALPHA = 0.490829
Minibatch loss at step 200: 3.303080
Minibatch accuracy: 85.2%
Validation accuracy: 83.1%
Test accuracy: 90.3%


----------------> ALPHA = 0.486329
Minibatch loss at step 300: 3.068282
Minibatch accuracy: 82.0%
Validation accuracy: 83.7%
Test accuracy: 90.5%


----------------> ALPHA = 0.48187
Minibatch loss at step 400: 2.752303
Minibatch accuracy: 85.2%
Validation accuracy: 85.3%
Test accuracy: 92.0%


----------------> ALPHA = 0.477452
Minibatch loss at step 500: 2.467303
Minibatch accuracy: 89.8%
Validation accuracy: 85.9%
Test accuracy: 92.5%


----------------> ALPHA = 0.473075
Minibatch loss at step 600: 2.452021
Minibatch accuracy: 83.6%
Validat